# Analysis of hover points

In [ ]:
%matplotlib notebook

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import pandas as pd

import glob
import os

In [ ]:
# global plotting settings
markersize = 3
cmap = cm.cool

In [ ]:
# declare ALL list objects necessary

current = []
voltage = []
rpm = []
p_motor, p_motorpair, p_upper, p_lower  = [], [], [], []
in_throttle = []
out_throttle = []
nsh_cmd = []

time = []

p_total = []
p_upper_total = []
p_lower_total = []

in_throttle_av = []
p_motor_av = []
p_motorpair_av = [] 
p_total_av = []
p_upper_av = []
p_lower_av = []
p_upper_total_av = []
p_lower_total_av = []
delta_rpm = []

x = []
y = []
z = []
u = []
v = []
w = []
p = []
q = []
r = []

dt = []
dy = []
dx = []
dz = []

v_norm = []
dv_norm = []

v_limit = []
dv_limit = []

v_filtered = []
dv_filtered = []

sequence_v = []
sequence_dv = []

In [ ]:
# get the .csv files for the given velocity limit
def get_files_for_limit(limit, discard=True):
    
    v_limit = limit * 0.01*14.345219306215128
    data_directory = f'../flight_data/2022-07-14_ag_field/flight_1/hover_{limit}_limit'
    csv_files = glob.glob(os.path.join(data_directory, "*.csv"))
    sorted_files = []
    
    # sort sequences in ascending order
    for i in range(len(csv_files)):
        for e in csv_files:
            if f'_{i}_' in e:
                sorted_files.append(e)
                break
    
    # discard first and land sequence (vehicle on ground before takeoff and after landing)
    if discard:
        return sorted_files[1:-1]
    
    else:
        return sorted_files
    

In [ ]:
csv_files = get_files_for_limit(1.0)
csv_files_full = get_files_for_limit(10, False)

sequence = []
full_flight = pd.DataFrame()

# stack sequences
for file in csv_files:
    sequence.append(pd.read_csv(file))    

for file_full in csv_files_full:
    full_flight = pd.concat([full_flight, pd.read_csv(file_full)])

In [ ]:
csv_files

In [ ]:
for i in range(0, len(sequence)):
    print(sequence[i].shape[0])

In [ ]:
flight_data = sequence[1]
#flight_data = full_flight
flight_data.rename(columns = {'omega1':' omega1'}, inplace = True)

time = ((flight_data['t'].values - min(flight_data['t'].values)) / 10e5)

# get position and velocities
x = flight_data['x'].values
y = flight_data['y'].values
z = flight_data['z'].values

u = flight_data['u'].values
v = flight_data['v'].values
w = flight_data['w'].values

# get RPM, voltage and current
omega = [flight_data[f' omega{i}'] for i in range(1,9)]
voltage = [flight_data[f'U1{i}'] for i in range(1,9)]
current = [flight_data[f'I1{i}'] for i in range(1,9)]

v_norm = np.sqrt(u ** 2 + v ** 2 + w ** 2)

In [ ]:
# plot histogram of vnorm with corresponding limit

fig0, ax0 = plt.subplots(1, 2, figsize=(10,5))

# First flight
ax0[0].hist(v_norm, bins=20)
ax0[0].set_xlabel('v_norm [m/s]')
ax0[0].set_ylabel('counts')
ax0[0].set_title('Histogram of vnorm')
ax0[0].grid()

# Second flight
ax0[1].plot(time, v_norm, label='v_norm')
ax0[1].plot(time, u, label='u')
ax0[1].plot(time, v, label='v')
ax0[1].plot(time, w, label='w')
ax0[1].set_title('x,y,z velocity and v norm')
ax0[1].grid()
ax0[1].legend()
ax0[1].set_xlim(min(time),max(time))

In [ ]:
# rpm over time

fig1, ax1 = plt.subplots(2, 2, figsize=(10,8))

# Rotor pairs

ax1[0][0].plot(time,omega[1], label='rotor2')
ax1[0][0].plot(time,omega[4], label='rotor5')

ax1[0][1].plot(time,omega[0], label='rotor1')
ax1[0][1].plot(time,omega[5], label='rotor6')


ax1[1][0].plot(time,omega[2], label='rotor3')
ax1[1][0].plot(time,omega[7], label='rotor8')

ax1[1][1].plot(time,omega[3], label='rotor4')
ax1[1][1].plot(time,omega[6], label='rotor7')


for i in range(2):
    for j in range(2):
        ax1[i][j].set_xlabel('time')
        ax1[i][j].set_ylabel('AngVel')
        ax1[i][j].set_ylim(1900,2500)
        ax1[i][j].grid()
        ax1[i][j].legend()

ax1[0][0].set_title(f'RPM Motor pair 2-5')
ax1[0][1].set_title(f'RPM Motor pair 1-6')
ax1[1][0].set_title(f'RPM Motor pair 3-8')
ax1[1][1].set_title(f'RPM Motor pair 4-7')
fig1.tight_layout()

In [ ]:
# current over time

fig1, ax1 = plt.subplots(2, 2, figsize=(10,8))

# Rotor pairs

ax1[0][0].plot(time,current[1], label='rotor2')
ax1[0][0].plot(time,current[4], label='rotor5')

ax1[0][1].plot(time,current[0], label='rotor1')
ax1[0][1].plot(time,current[5], label='rotor6')


ax1[1][0].plot(time,current[2], label='rotor3')
ax1[1][0].plot(time,current[7], label='rotor8')

ax1[1][1].plot(time,current[3], label='rotor4')
ax1[1][1].plot(time,current[6], label='rotor7')


for i in range(2):
    for j in range(2):
        ax1[i][j].set_xlabel('time')
        ax1[i][j].set_ylabel('AngVel')
        ax1[i][j].grid()
        ax1[i][j].legend()

ax1[0][0].set_title(f'Current Motor pair 2-5')
ax1[0][1].set_title(f'Current Motor pair 1-6')
ax1[1][0].set_title(f'Current Motor pair 3-8')
ax1[1][1].set_title(f'Current Motor pair 4-7')
fig1.tight_layout()

In [ ]:
# current over time

fig1, ax1 = plt.subplots(2, 2, figsize=(10,8))

# Rotor pairs

ax1[0][0].plot(time,voltage[1], label='rotor2')
ax1[0][0].plot(time,voltage[4], label='rotor5')

ax1[0][1].plot(time,voltage[0], label='rotor1')
ax1[0][1].plot(time,voltage[5], label='rotor6')


ax1[1][0].plot(time,voltage[2], label='rotor3')
ax1[1][0].plot(time,voltage[7], label='rotor8')

ax1[1][1].plot(time,voltage[3], label='rotor4')
ax1[1][1].plot(time,voltage[6], label='rotor7')


for i in range(2):
    for j in range(2):
        ax1[i][j].set_xlabel('time')
        ax1[i][j].set_ylabel('AngVel')
        ax1[i][j].grid()
        ax1[i][j].legend()

ax1[0][0].set_title(f'Voltage Motor pair 2-5')
ax1[0][1].set_title(f'Voltage Motor pair 1-6')
ax1[1][0].set_title(f'Voltage Motor pair 3-8')
ax1[1][1].set_title(f'Voltage Motor pair 4-7')
fig1.tight_layout()